In [7]:
brew install lightgbm


SyntaxError: invalid syntax (<ipython-input-7-0eadaa3233a2>, line 1)

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

OSError: dlopen(/Users/calumthompson/opt/anaconda3/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/calumthompson/opt/anaconda3/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

# Explore the data set - current season

## Player performance this season

In [ ]:
player_season_sum_address = '/Users/calumthompson/Documents/Fantasy football/Fantasy-Premier-League/data/2019-20/cleaned_players.csv'
player_season_sum = pd.read_csv(player_season_sum_address)

In [ ]:
player_season_sum.head(10)

In [ ]:
player_season_sum.loc[(player_season_sum['first_name'] == 'Jamie') & (player_season_sum['second_name'] == 'Vardy')]

## week by week player performance

- name - forename_surname_### (not sure what number means)

- GW - Game week

- total_points - marginal points for given GW

- Value - 10x cost on FPL


In [ ]:
player_GW_address = '/Users/calumthompson/Documents/Fantasy football/Fantasy-Premier-League/data/2019-20/gws/merged_gw.csv'
player_GW = pd.read_csv(player_GW_address)

In [ ]:
player_GW.sort_values(['name','GW']).head(100)

In [ ]:
player_GW.loc[player_GW['name'] =='Jamie_Vardy_166'][['GW','total_points']].plot.scatter(x = 'GW', y = 'total_points')

# Point Acquisition Stochastic System (PASS)

## Import data

In [ ]:
Season_GW_address = '/Users/calumthompson/Documents/Fantasy football/Fantasy-Premier-League/data/2018-19/gws/merged_gw.csv'
Last_season_totals_address = '/Users/calumthompson/Documents/Fantasy football/Fantasy-Premier-League/data/2017-18/cleaned_players.csv'

### 2018/19 GW data

In [ ]:
Season_GW = pd.read_csv(Season_GW_address, encoding='latin-1')

In [ ]:
Season_GW['Forename'] = Season_GW['name'].str.split('_', expand=True)[0]
Season_GW['Surname'] = Season_GW['name'].str.split('_', expand=True)[1]

In [ ]:
Season_GW = Season_GW.sort_values(['Forename','Surname','GW'])
Season_GW.loc[Season_GW['GW'] > 1,'last_weeks_points'] = Season_GW['total_points'].shift(periods = 1)
Season_GW.loc[Season_GW['GW'] > 2,'last_2weeks_points'] = Season_GW['total_points'].shift(periods = 2)
Season_GW.loc[Season_GW['GW'] > 3,'last_3weeks_points'] = Season_GW['total_points'].shift(periods = 3)
Season_GW.loc[Season_GW['GW'] > 4,'last_4weeks_points'] = Season_GW['total_points'].shift(periods = 4)

### 2017/18 season totals

In [ ]:
Last_season_totals = pd.read_csv(Last_season_totals_address, encoding='latin-1')

In [ ]:
Last_season_totals = Last_season_totals[['first_name', 'second_name','total_points', 'minutes', 'goals_scored', 'goals_conceded', 'selected_by_percent', 'clean_sheets','red_cards','yellow_cards']]

Last_season_totals.columns = ['Forename','Surname',
                              'last_season_total_points', 'last_season_total_minutes', 
                              'last_season_goals_scored', 'last_season_goals_conceded',
                              'last_season_selected_by_percent', 'last_season_clean_sheets',
                              'last_season_red_cards', 'last_season_yellow_cards']


### Merge

In [ ]:
Analysis_df = pd.merge(Season_GW,Last_season_totals, how = 'left', left_on = ['Forename','Surname'], right_on = ['Forename','Surname'])

## Sense check

### 2017/18 top point earners

In [ ]:
Analysis_df.groupby(['Surname','Forename']).agg({'last_season_total_points': np.mean, 
                                                 'last_season_goals_scored' :np.mean}
                                                ).sort_values('last_season_total_points', ascending = False).head(20)

### 2018/19 top point earners

In [ ]:
Analysis_df.groupby(['Surname','Forename']).agg({'total_points': sum, 
                                                 'goals_scored' :np.mean}
                                                ).sort_values('total_points', ascending = False).head(20)

In [ ]:
plt.hist(x = Analysis_df['last_season_total_points'], cumulative=True, bins = 100, histtype = 'step', normed = True)
plt.show()

# Model

In [ ]:
LGB_PARAMS = {
    'learning_rate': 0.063,
    'max_depth': 5,
    'num_leaves': 40, 
    'objective': 'binary',
    'metric':'binary_logloss',
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'max_bin': 100,
    'seed':123,}


def crossvalidate_lgb(X_train, y_train, lgb_params, num_boost_round=2000, nfold=5):
    # form LightGBM datasets
    dtrain_lgb = lgb.Dataset(X_train, label=y_train)
    # LightGBM, cross-validation
    cv_result_lgb = lgb.cv(lgb_params, 
                           dtrain_lgb, 
                           num_boost_round=num_boost_round, 
                           nfold=nfold, 
                           stratified=True, 
                           early_stopping_rounds=50, 
                           verbose_eval=10, 
                           show_stdv=True,
                          seed=123,)
    num_boost_rounds_lgb = len(cv_result_lgb['binary_logloss-mean'])
    print(f'num_boost_rounds_lgb={num_boost_rounds_lgb}')
    # train model
    model_cv = lgb.LGBMClassifier(n_estimators=num_boost_rounds_lgb, **lgb_params)
    model_cv.fit(X_train,y_train)
    return cv_result_lgb, model_cv

In [ ]:
y = Analysis_df['total_points']
X = Analysis_df.drop(columns = 'total_points')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
crossvalidate_lgb (X_train, y_train, lgb = LGB_PARAMS)